# Automated generation of a PMIC Core for SKY130

> &copy; AC3E Microelectronics Team, 2024, SPDX-License-Identifier: LGPL-3.0-or-later
</br>

#### Team Members 

|Name|Affiliation|IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|
| Jorge Marín (Team Coordinator, Postdoctoral Fellow) <br /> Email ID: jorge.marinn@usm.cl|AC3E (Chile)| Yes |Yes|
| Christian Rojas (Professor Advisor) <br /> Email ID: c.a.rojas@ieee.org|AC3E, Universidad Técnica Federico Santa María (Chile)| Yes |No|
| Daniel Arevalos (Master Student) <br /> Email ID: daniel.arevalos@sansano.usm.cl|AC3E, Universidad Técnica Federico Santa María (Chile)| Yes |No|
| Mario Romero (Undergraduate Student) <br /> Email ID: mario.romeron@usm.cl|AC3E, Universidad Técnica Federico Santa María (Chile)| Yes |No|

## Abstract

This Jupyter Notebook aims to automate the design of a DC-DC
converter power stage and a series low dropout (LDO) linear voltage regulator targeting the highly efficient generation of a parametrized regulated supply voltage from a given power source (e.g. 5V to 3V DC-DC conversion). This includes the automatic layout generation of both building blocks on the system specifications. The power supply chain has been designed using the Skywater 130nm technology and open-source design tools. Firstly, the power stage is realized using an automated flow which
considers the output current, the output voltage, the operating frequency and the maximum area as main constraints entered by the user. Afterwards, the LDO is designed based on the provided specifications: 

## Introduction

Power management is an unavoidable concern in application-specific integrated circuits (ASIC) design projects. The importance of power considerations gets even more critical when we consider future edge devices for the Internet of Things (IoT), such as an energy-autonomous node with energy harvesting capabilities and analog, mixed signal and digital on-chip function blocks.
The most challenging part of the power management system are the blocks downstream towards the function blocks, namely the DC-DC converters and the LDOs (Fig. 1). In previous works, this problem has been considered by eliminating the LDO and replacing it by an active filter [1] or by considering different DC-DC-LDO on-chip or off-chip configurations, depending on the operation requirements.
In this Jupyter Notebook, we focus on the on-chip realization of the DC-DC-LDO series connection, centering our analysis on the generation of a custom design of an efficient and compact PMIC core using the Skywater 130nm (SKY130) open-source CMOS technology.

![](https://media.githubusercontent.com/media/pmicgen/pmicgen/main/.github/figures/block_diagram.png)

[Fig. 1: Block diagram of the DC-DC in series with an LDO system studied in this work [3].]

This notebook aims to implement the followng innovations:

1. A simplified implementation of a LUT-based LDO automated design proposed in [4] using the SKY130 technology
2. The integration of existing IP building blocks designed by the opensource community into a unified flow for the specs-to-GDS generation of an LDO
3. The integration and use of layout automation tools/flows (such as ALIGN and GDSfactory) to produce DRC-verified layout of the LDO block within a Jupyter notebook
4. Simulation and design exploration of the generated LDO [WORK IN PROGRESS]
5. The automated design of a DC-DC conversion power stage, based on our previous work [5] [WORK IN PROGRESS]
6. The integration of the PMIC core consisting of a DC-DC-LDO connection into a unified flow for spec-to-GDS generation and design space exploration [WORK IN PROGRESS]


## PMIC core operating principle

Robust power delivery systems must ensure stable voltages at specified load currents, incorporating sufficient regulation capabilities to enable the reliable operation of digital, analog, mixed-signal, and radio-frequency (RF) function blocks. In modern applications, the full integration of these solutions result in compact and low-cost systems with high efficiency and robustness.

Traditionally, power management integrated circuits (PMICs) employ two types of regulators: DC-DC converters (also known as switching regulators) and low-dropout (LDO) linear regulators. Efficient switching DC-DC converters allow large input-output conversion ratios at the cost of large chip area and considerable output ripple, generating the need to use large passive components which are not compatible with integration in order to mitigate this effect. Placing these components off chip decreases the efficiency due to parasitic capacitances and power losses introduced by packaging, bonding and PCB traces. On the other hand, compact linear regulators can achieve low ripple at the expense of low efficiency, which degrades as the dropout voltage increases.

Lately, heterogeneous integration approaches have been proposed with series and cascaded DC-DC and LDO blocks to generate multiple regulated voltage domains [2]. DC-DC circuits are placed close to the energy source to perform voltage conversion using large ratios. Complementary, LDOs are placed locally, close to the function blocks, to provide regulated voltage with a low dropout to enhance the efficiency of the power management system, allowing supply-sensitive circuits to operate correctly [6].

Design automation is a powerful tool which can help to decrease the design effort of complex integrated circuits in a myriad of scenarios. In the case of power circuits, a holistic analysis of efficiency and interference in a multi-channel regulated voltage network and a subsequent synthesis of the physical implementation based on a set of specifications can be of great help in the design of high-performance and low-power ASICs. This is the final aim of our current ongoing work.

## About the notebook
The presented Jupyter Notebook is summarized in Fig. 2.

![](https://media.githubusercontent.com/media/pmicgen/pmicgen/main/.github/figures/flow.svg)

[Fig. 2: This Jupyter Notebook's block diagram.]

## 1. Tools and dependencies

The logic used in this project is contained in the `pmicgen` command line interface utility, which can be installed with using `pip install` according to the `setup.py` package settings. Aside from this tool is necessary some open-source IC development tools to run simulations and view reports generated by the `pmicgen` utility. This extra set of tools are installed via `conda` through the `environment.yml` file.



### 1.1 Google Colab Installation
To install the conda environment in Google Colab execute the following code block.


In [ ]:
if 'google.colab' in str(get_ipython()):
    # Removes CFFI to avoid future conflicts between condacolab's package
    # and the debian CFFI precompiled binaries that comes with Colab
    %pip uninstall -q -y cffi
    %pip install -q condacolab
    import condacolab
    condacolab.install()

> The Python kernel needs to be restarted for changes to be applied. This happens automatically.
If you are wondering why you are seeing a message saying "Your session crashed for an unknown reason", this is why. 
You can safely ignore this message!

Now you can clone the repository and install `pmicgen` running the next code block:

> Please be patient, the installation takes about 15 minutes in Colab

In [ ]:
# Clone the precompiled optimization library interface to reduce the total installation time
%cd /content
!git clone --quiet https://github.com/ALIGN-analoglayout/ILPSolverInterface
%env ALIGN_ILP_PATH=/content/ILPSolverInterface/manylinux_2_28_x86_64

# Clone the pmicgen repository and extract the look-up table
!git clone --quiet --recursive https://github.com/pmicgen/pmicgen
!git clone https://github.com/efabless/mpw_precheck
%cd /content/pmicgen
!unzip -q analysis/sky130A_LUT.npz

# Install pmicgen and its environment
!mamba env update -q -n base -f env/environment.yml
%pip install -q --progress-bar off .

> The step above can generate warnings about core dependencies being updated, Colab suggests a restart though a pop-up, however this warning can be safely ignored and cancelled, just make sure the installation is finished if you select to restart the environment.

After the environment is set up, the SKY130 PDK can be installed though volare.

In [ ]:
%env PDK_ROOT=/root/.volare
%env PDK=sky130A
# Install the latest SKY130 PDK version
!volare enable $(volare ls-remote | sed -n '1 p')

### 1.2 Local Installation

> This step does not need to be ran in Colab

> Warning: The recommended usage for now is Google Colab, the docker installation may have some broken paths.

For the local installation follow the instructions in the README to setup the provided jupyter server. This server has a custom kernel with the conda environment already installed. Then you can installed `pmicgen` running the next block.

In [ ]:
%cd /home/jovyan
%pip install -q --user .

### 1.3 Verification

After installing the required tools, verify the `pmicgen` functionality running a sample command.

In [ ]:
!pmicgen --help

## 2. User specs for the LDO

Change the following variables accordingly to generate the internal LDO parameters.

In [ ]:
def get_user_input(prompt, default):
    """
    Asks the user to enter a value or use the default.
    
    :param prompt: The prompt message to display to the user.
    :param default: The default value.
    :return: The user input or the default value.
    """
    user_input = input(f"{prompt} [Default: {default}] (leave blank to use default): ").strip()
    return default if user_input == "" else type(default)(user_input)

print("LDO Configuration\n")

# Define default values
defaults = {
    'Vreg': 1.2,      # LDO output voltage
    'Vdd': 1.8,       # LDO supply voltage
    'Vref': 0.8,      # LDO voltage reference
    'R1': 100000,     # Resistance
    'iq': 1.5e-6,     # Current through the OTA
    'il': 1e-3,       # Load current
    'cl': 5e-12       # Load capacitance
}

# Dictionary to hold user choices
user_choices = {}

# Ask for each parameter
for param, default in defaults.items():
    user_choice = get_user_input(f"Enter {param} or use default", default)
    user_choices[param] = user_choice

user_choices['R2'] = int(user_choices['R1']/((user_choices['Vreg']/user_choices['Vref'])-1))

print("\nConfiguration Summary:")
for param, value in user_choices.items():
    print(f"{param}: {value}")

psr_condition = -46
load_regulation_condition = "min"
phase_margin_condition = 94
size_condition = "min"

from analysis.ldo import LDO

ldo = LDO(psr_condition, load_regulation_condition, phase_margin_condition, size_condition)

# 3. Specs to device size for the LDO

## 3.1 LUT generation

In the figure below the simplified design flow in [4] adapted to the SKY130 technology is depicted. This integrates the Modified Modal Analysis algorithm with the pre-generated LUT technique using the gm/id approach.

The script from https://github.com/medwatt/gmid.git was modified to work correctly with the open source pdk.

![](https://media.githubusercontent.com/media/pmicgen/pmicgen/main/.github/figures/spectodevsize.png)

In [ ]:
from mosplot import LookupTableGenerator
from pathlib import Path


obj = LookupTableGenerator(
    description="freepdk sky130 ngspice",
    simulator="ngspice",
    model_paths=[
        f"{Path.home()}/.volare/sky130A/libs.tech/ngspice/sky130.lib.spice tt",
        ],
    model_names={
        "pmos": "sky130_fd_pr__pfet_01v8_lvt",
        "nmos": "sky130_fd_pr__nfet_01v8_lvt",
    },
    vsb=(0, 1.8, 0.1),
    vgs=(0, 1.8, 0.01),
    vds=(0, 1.8, 0.01),
    width=1e-06,
    lengths=[0.4e-06, 0.8e-06, 1.6e-06, 3.2e-06, 6.4e-06],
)

The look-up table generation is disabled since it can take a long time to generate.
A pre-built version of this LUT is included in the repository so it does not need to be regenerated

## 3.2 OTA operation point calculation

### 3.2.1 Simulation netlist generation

Here you need to specify your OTA design and the simulation circuit.

In [ ]:
%cd /content/pmicgen

from analysis.ota_op import template_generator
import subprocess
from analysis.utils import *
from analysis.ldo_small_signal_modeling import small_signal_macromodel, small_signal_device
from pathlib import Path

ota_name = "OTA1st_lvt_jm"
ota_netlist_path = "./xschem/designs/ota1st_lvt_jm/OTA1st_lvt_jm.spice"
netlist_output = "/tmp/OTA_op_netlist.spice"
op_output_data = "op_data"

template = template_generator(ota_name=ota_name,
                              output_file_path=op_output_data,
                              model_paths=[f".lib {Path.home()}/.volare/sky130A/libs.tech/ngspice/sky130.lib.spice tt"],
                              device_params_instantiation_model= "m{device_model}",
                              simulation_circuit = ["V1 V3V3 GND 1.8", 
                                                    "I0 GND net1 1.5u", 
                                                    "V2 net2 GND {CM_VOLTAGE}", 
                                                    "V3 INP net2 AC 1", 
                                                    "C1 OUT GND 1f m=1", 
                                                    "R1 net3 INM 10E6 m=1", 
                                                    "V4 OUT net3 {OUTPUT_VOLTAGE-CM_VOLTAGE}", 
                                                    "C2 INM GND 1 m=1"],
                              subckt_instantation = "x1 INP INM OUT V3V3 GND net1 {subckt}",
                              ota_netlist_path=ota_netlist_path,
                              netlist_output=netlist_output)

template.build()

### 3.2.2 Run the simulation to get the operation point parameters of every transistor

In [ ]:
print(f"run ngspice")
ngspice_command = f"ngspice -b {template.netlist_output}"
subprocess.run(ngspice_command, shell=True)

### 3.2.3 Parse the simulation data and change node names for numbers

In [ ]:
op_data = op_parser(template)
op_data

## 3.3 Modified Modal Analysis (Symbolic Solve)


### 3.3.1 Closed-Loop Macromodel Generation

In [ ]:
nodes, in_pos_node, next_node = node_identification(template)
ldo_output_node = next_node
nodes

In [ ]:
ldo_output_node

In [ ]:
macromodel = small_signal_macromodel(macromodel_file_path = "/tmp/ldo_macromodel.spice")
small_signal_devices = macromodel.build(template, op_data, nodes)

f = open(macromodel.macromodel_file_path, "a")
f.write(f"Vdd {nodes['VDD']} {nodes['VSS']} 1\n")
f.write(f"Vref {nodes['IN_M']} {nodes['VSS']} 1\n")
pass_tranistor = small_signal_device(name="pt", 
                                                gds=1, 
                                                gm=1,
                                                cgs=1, 
                                                cgd=1, 
                                                vs=nodes["VDD"], vd=ldo_output_node, vg=nodes["OUT"])
print("\n".join(pass_tranistor.get_model_spice()))
f.write("\n".join(pass_tranistor.get_model_spice()))
f.write(f"\nR1 {nodes['IN_P']} {ldo_output_node} 100000\n")
f.write(f"R2 0 {nodes['IN_P']} 200000\n")
f.write(f"C_load 7 0 1e-16\n")
f.close()

f = open(macromodel.macromodel_file_path, "r")
print(f.read())
f.close()

### 3.3.2 Open-Loop Macromodel Generation

In [ ]:
break_node = "IN_P"
node_disp = next_node+1

nodes, in_pos_node, next_node = node_identification_openloop(template)
ldo_output_node = next_node
nodes

In [ ]:
macromodel = small_signal_macromodel(macromodel_file_path = "/tmp/ldo_macromodel_openloop.spice")
small_signal_devices = macromodel.build(template, op_data, nodes)

ldo_break_node = ldo_output_node+1

f = open(macromodel.macromodel_file_path, "a")
f.write(f"Vdd {nodes['IN_P']} {nodes['VSS']} 1\n")
pass_tranistor = small_signal_device(name="pt", 
                                                gds=1, 
                                                gm=1,
                                                cgs=1, 
                                                cgd=1, 
                                                vs=nodes["VDD"], vd=ldo_output_node, vg=nodes["OUT"])
print("\n".join(pass_tranistor.get_model_spice()))
f.write("\n".join(pass_tranistor.get_model_spice()))
f.write(f"\nR1 {ldo_break_node} {ldo_output_node} 100000\n")
f.write(f"R2 0 {ldo_break_node} 200000\n")
f.write(f"C_load 7 0 1e-16\n")
f.close()

f = open(macromodel.macromodel_file_path, "r")
print(f.read())
f.close()

### 3.3.3 MNA 

Closed loop MNA

In [ ]:
from analysis.symbolic_mna import symbolic_mna
closed_loop_sym_mna = symbolic_mna()
closed_loop_sym_mna.netlist = "/tmp/ldo_macromodel.spice"
A = closed_loop_sym_mna.build()

Open loop MNA

In [ ]:
from analysis.symbolic_mna import symbolic_mna    
open_loop_sym_mna = symbolic_mna()
open_loop_sym_mna.netlist = "/tmp/ldo_macromodel_openloop.spice"
B = open_loop_sym_mna.build()

## 3.4 Pass Transistor Exploration

Tests and store the pass transistor characteristics

In [ ]:
from mosplot import load_lookup_table
import numpy as np
import analysis.ldo_mna as mna
import matplotlib.pyplot as plt
import sympy as sym

lookup_table = load_lookup_table("sky130A_LUT.npy")
## LDO exploration parameters (this vakues can be change as the user see fit)
lengths = [0.4e-06, 0.6e-06, 0.8e-06, 1.6e-06]      # Length exploration values.
gmid_sweep = np.arange(5,25,0.2)               # gm/id sweep for exploration.
pass_transistor = mna.pass_transistor_exploration(lookup_table, user_choices['Vdd'], user_choices['Vreg'], user_choices['il'], user_choices['R1'], user_choices['R2'], lengths, gmid_sweep, (-1.8, -0.1, 0.1))
"""
fig, axs = plt.subplots(2, 3, figsize=(12, 6))
fig.suptitle('Pass Transistor Characteristics')
for index, val in enumerate(lengths):
    axs[0, 0].plot(gmid_sweep, pass_transistor.vgs[index,:])
axs[0, 0].set_title('Vgs vs gmid')
for index, val in enumerate(lengths):
    axs[0, 1].plot(gmid_sweep, pass_transistor.Jd[index,:])
axs[0, 1].set_title('Jd vs Vgs')
for index, val in enumerate(lengths):
    axs[0, 2].plot(gmid_sweep, pass_transistor.gm[index,:])
axs[0, 2].set_title('gm vs vgs')
for index, val in enumerate(lengths):
    axs[1, 0].plot(gmid_sweep, pass_transistor.gds[index,:])
axs[1, 0].set_title('gds vs Vgs')
for index, val in enumerate(lengths):
    axs[1, 1].plot(gmid_sweep, pass_transistor.W[index,:])
axs[1, 1].set_title('W vs Vgs')
for index, val in enumerate(lengths):
    axs[1, 2].plot(gmid_sweep, pass_transistor.cgd[index,:])
axs[1, 2].set_title('cgd vs Vgs')
axs[1,1].set_yscale('log')
"""

## 3.5 Design Space Exploration
Following the previous derivations of the circuit parameters in this section we sweep the relevants parameters to explore the design space of the LDO block

### 3.5.1 PSR

In [ ]:
L = sym.sympify("L")
W = sym.sympify("W")

Area = sym.lambdify([L, W], L+W)   # Total area of the pass transistor

lengths_reshaped = np.ndarray.flatten(np.transpose(np.tile(lengths, (100,1))))
f_2 = Area(lengths_reshaped, np.ndarray.flatten(pass_transistor.W))

area_mask = []
for x in f_2:
    if (x<ldo.size_condition):
        area_mask.append(True)
    else:
        area_mask.append(False)

In [ ]:
from sympy import *
import cmath as math

components_values = closed_loop_sym_mna.components_values()
s=Symbol('s')
components_values[sympify('Gm_pt')]=sympify('Gm_pt')
components_values[sympify('Rds_pt')]=sympify('Rds_pt')
components_values[sympify('Cgs_pt')]=sympify('Cgs_pt')
components_values[sympify('Cgd_pt')]=sympify('Cgd_pt')
components_values[sympify('Vref')]=0

ldo_sym_equation = closed_loop_sym_mna.mna_equation()
ldo_num_equation = ldo_sym_equation.subs(components_values)
ldo_num_equation = ldo_num_equation.subs({s:0})
ldo_num_equation

In [ ]:
ldo_num_eq_solve = solve(ldo_num_equation,closed_loop_sym_mna.X)
ldo_output_tf = ldo_num_eq_solve[closed_loop_sym_mna.X[ldo_output_node]]
ldo_output_tf

In [ ]:
Gm_pt = sym.Symbol('Gm_pt')
Rds_pt = sym.Symbol('Rds_pt')

ldo_psr_dc_eq_lamb = sym.lambdify([Gm_pt, Rds_pt], ldo_output_tf)
ldo_psr_dc = ldo_psr_dc_eq_lamb(np.multiply(pass_transistor.gm, pass_transistor.W*1e6), 1/np.multiply(pass_transistor.gds, pass_transistor.W*1e6))
ldo_psr_dc_db = 20*np.log10(np.abs(ldo_psr_dc))

ldo_psr_dc_db = np.ndarray.flatten(ldo_psr_dc_db)
ldo_psr_dc_db_mask = []
for x in ldo_psr_dc_db:
    if(x<ldo.psr_condition):
        ldo_psr_dc_db_mask.append(True)
    else:
        ldo_psr_dc_db_mask.append(False)

L = sym.Symbol('L')
W = sym.Symbol('W')

f_1 = ldo_psr_dc_db
plt.scatter(f_2,f_1)
plt.scatter(f_2[np.asarray(ldo_psr_dc_db_mask) & np.asarray(area_mask)], f_1[np.asarray(ldo_psr_dc_db_mask) & np.asarray(area_mask)])
plt.legend(['Nominal', 'PSR < '+str(psr_condition)])
plt.xscale('log')
plt.title('PSR vs Area')
plt.xlabel('Area')
plt.ylabel('PSR')
plt.xlim([10**-5,10**-2])

### 3.5.2 Phase Margin

In [ ]:
from sympy import *
import cmath as math

components_values = open_loop_sym_mna.components_values()
components_values[sympify('Gm_pt')]=sympify('Gm_pt')
components_values[sympify('Rds_pt')]=sympify('Rds_pt')
components_values[sympify('Cgs_pt')]=sympify('Cgs_pt')
components_values[sympify('Cgd_pt')]=sympify('Cgd_pt')

eq = open_loop_sym_mna.mna_equation()
eq1 = eq.subs(components_values)
u1 = solve(eq1,open_loop_sym_mna.X)
G = u1[open_loop_sym_mna.X[ldo_output_node-1]]
G

In [ ]:
# 1. Lambdify the matrix so the parameters of the Pass Transistor can be replaced
s = Symbol('s')
Gm_pt = sym.Symbol('Gm_pt')
Rds_pt = sym.Symbol('Rds_pt')
Cgs_pt = sym.Symbol('Cgs_pt')
Cgd_pt = sym.Symbol('Cgd_pt')
PSRR_DC_lamb = sym.lambdify([Gm_pt, Rds_pt, Cgs_pt, Cgd_pt], G)

psr = PSRR_DC_lamb(np.multiply(pass_transistor.gm, pass_transistor.W*1e6),
                   1/np.multiply(pass_transistor.gds, pass_transistor.W*1e6),
                   np.multiply(np.abs(pass_transistor.cgs), pass_transistor.W*1e6),
                   np.multiply(np.abs(pass_transistor.cgd), pass_transistor.W*1e6))

def get_pm(psr, s_sweep):
    
    bode_data = []
    phase_data = []
    for idx, value in enumerate(s_sweep):
            PSRR = psr.subs({s:2*math.pi*value*1j})
            magnitude = 20*np.log10(float(np.abs(PSRR)))
            bode_data.append(magnitude)
            phase_data.append(math.phase(PSRR))
    return bode_data, phase_data

pm = []
s_sweep = np.logspace(1,10, num=100, base=10)
for eq_l in psr:
    for eq_w in eq_l:
        bode_data, phase_data = get_pm(eq_w, s_sweep)
        ugf = np.argmin(np.abs(bode_data))
        tmp = 180+phase_data[ugf]*180/math.pi
        pm.append(tmp)

> Warning: This step can take a long time (~25min in Colab)

In [ ]:
pm_mask = []
for x in pm:
    if(x>ldo.phase_margin_condition):
        pm_mask.append(True)
    else:
        pm_mask.append(False)
pm = np.asarray(pm)

In [ ]:
plt.scatter(f_2, pm)
plt.scatter(f_2[np.asarray(pm_mask) & np.asarray(area_mask)], pm[np.asarray(pm_mask) & np.asarray(area_mask)])
plt.xscale('log')
plt.xlim([10**-5,10**-2])
plt.ylim([0,150])
plt.title('Phase margin')
plt.xlabel('Pass transistor Area')

### 3.5.3 Load Regulation

In [ ]:
from sympy import *
import cmath as math

In [ ]:
components_values = closed_loop_sym_mna.components_values()
components_values[sympify('Gm_pt')]=sympify('Gm_pt')
components_values[sympify('Rds_pt')]=sympify('Rds_pt')
components_values[sympify('Cgs_pt')]=sympify('Cgs_pt')
components_values[sympify('Cgd_pt')]=sympify('Cgd_pt')
z_matrix = Matrix(closed_loop_sym_mna.Z)
z_matrix = z_matrix+Matrix([0,0,0,0,0,0,1.5e-6,-0.667e-3,0.8,-0.2])
load_reg_eq = Eq(closed_loop_sym_mna.A*Matrix(closed_loop_sym_mna.X),z_matrix)
load_reg_num_eq = load_reg_eq.subs(components_values)
load_reg_num_eq = load_reg_num_eq.subs({s:0})
load_reg_num_eq_solve = solve(load_reg_num_eq,closed_loop_sym_mna.X)
load_reg_output = load_reg_num_eq_solve[closed_loop_sym_mna.X[ldo_output_node]]
load_reg_output_lamb = sym.lambdify([Gm_pt, Rds_pt], load_reg_output)
load_reg_1 = load_reg_output_lamb(np.multiply(pass_transistor.gm, pass_transistor.W*1e6), 1/np.multiply(pass_transistor.gds, pass_transistor.W*1e6))
load_reg_1

In [ ]:
components_values = closed_loop_sym_mna.components_values()
components_values[sympify('Gm_pt')]=sympify('Gm_pt')
components_values[sympify('Rds_pt')]=sympify('Rds_pt')
components_values[sympify('Cgs_pt')]=sympify('Cgs_pt')
components_values[sympify('Cgd_pt')]=sympify('Cgd_pt')
z_matrix = Matrix(closed_loop_sym_mna.Z)
z_matrix = z_matrix+Matrix([0,0,0,0,0,0,1.5e-6,-0.667e-2,0.8,-0.2])
load_reg_eq = Eq(closed_loop_sym_mna.A*Matrix(closed_loop_sym_mna.X),z_matrix)
load_reg_num_eq = load_reg_eq.subs(components_values)
load_reg_num_eq = load_reg_num_eq.subs({s:0})
load_reg_num_eq_solve = solve(load_reg_num_eq,closed_loop_sym_mna.X)
load_reg_output = load_reg_num_eq_solve[closed_loop_sym_mna.X[ldo_output_node]]
load_reg_output_lamb = sym.lambdify([Gm_pt, Rds_pt], load_reg_output)
load_reg_2= load_reg_output_lamb(np.multiply(pass_transistor.gm, pass_transistor.W*1e6), 1/np.multiply(pass_transistor.gds, pass_transistor.W*1e6))
load_reg_2

In [ ]:
load_regulation = (load_reg_1-load_reg_2)/(0.667e-3-0.667e-2)
load_regulation = np.ndarray.flatten(load_regulation)

load_regulation_mask = []
for x in load_regulation:
    if(x<ldo.load_regulation_condition):
        load_regulation_mask.append(True)
    else:
        load_regulation_mask.append(False)

plt.scatter(f_2, load_regulation)
plt.scatter(f_2[np.asarray(load_regulation_mask) & np.asarray(area_mask)], load_regulation[np.asarray(load_regulation_mask) & np.asarray(area_mask)])
plt.xlabel('Pass transistor Area')
plt.title('Load Regulation')
plt.xlim([10**-5,10**-2])
plt.xscale('log')

### 3.5.3 Final Graphs

In [ ]:
import pandas as pd
import paretoset as pareto
final_mask = np.asarray(ldo_psr_dc_db_mask) & np.asarray(area_mask) & np.asarray(pm_mask) & np.asarray(load_regulation_mask)

opt_dict = {}
sense_list = []
for item in ldo.optimize:
    if item=="psr_condition":
        opt_dict[item]=f_1[final_mask]
        sense_list.append(ldo.optimize[item])
    elif item=="phase_margin_condition":
        opt_dict[item]=pm[final_mask]
        sense_list.append(ldo.optimize[item])
    elif item=="load_regulation_condition":
        opt_dict[item]=load_regulation[final_mask]
        sense_list.append(ldo.optimize[item])
    elif item=="size_condition":
        opt_dict[item]=f_2[final_mask]
        sense_list.append(ldo.optimize[item])


opt = pd.DataFrame(opt_dict)
mask = pareto.paretoset(opt, sense=sense_list)

In [ ]:
import paretoset as pareto
import pandas as pd


pareto_frontier = opt[mask]

plt.scatter(f_2,f_1)
plt.scatter(f_2[final_mask], f_1[final_mask])
plt.scatter(f_2[final_mask][mask], f_1[final_mask][mask])

plt.xscale('log')
plt.title('PSRR_DC')
plt.ylabel('PSRR')
plt.xlabel('Pass transistor Area')
plt.legend(['nominal', 'PM > 125', 'paretos frontier'])
print(f_1[final_mask][mask])


In [ ]:
selected_index = 4
print(np.ndarray.flatten(pass_transistor.W)[final_mask][mask][selected_index])
print(lengths_reshaped[final_mask][mask][selected_index])

## 4. LDO

To demonstrate the complete flow, each sub-block of the LDO such as the Bandgap Reference, Error Amplifier, Common Centroid Resistor and the PMOS device, will be built and tested separately

## 4.1. Bandgap Reference

The Bandgap reference will provide a fixed voltage at one differential input of the error amplifier.
Since this block is an essential element in the analog space and can be complex to build procedually by scratch, this block comes from an existing IP.
The current IPs for SKY130 to be choosen from are:
- https://github.com/johnkustin/bandgapReferenceCircuit
- https://github.com/mabrains/Analog_blocks/tree/main/Analog_Blocks/Bandgap (Option to choose this IP is in Work in Progress)

> A future feature is to benchmark and choose automatically the preferred bandgap from the IP list.

### 4.1.1 Generation
The following command outputs the design files at `build/sky130_bgr`

In [ ]:
%cd /content/pmicgen/xschem
!xschem -q -r -n --rcfile xschemrc tests/tb_bgr.sch
%cd /content/pmicgen
!pmicgen --tech sky130A bgr
!mkdir -p build/sky130_bgr/ngspice/pre_tb
!mv xschem/bgr.csv build/sky130_bgr/ngspice/pre_tb

### 4.1.2 Layout
This blocks shows the previously generated layout.

In [ ]:
%cd /content/pmicgen
import gdsfactory as gf
import sky130
bgr: gf.Component = gf.read.import_gds("build/sky130_bgr/gds/bgr.gds")
bgr.plot()


### 4.1.3 Schematic Simulation

Transient Simulation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_bgr/ngspice/pre_tb/bgr.csv',
    delim_whitespace=True,
    header=None,
    names=['t', 'v'])
plt.figure(figsize=(10, 6))
plt.plot(df['t'], df['v'], linestyle='-')
plt.title('Transient output')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.ylim([0, 2.3])
plt.grid(True)
plt.show()

## 4.2 Error Amplifier

The error amplifier is an OTA that comes from an input netlist and the layout its generated through the ALIGN analog netlist to layout flow.

### 4.2.1 Generation

The following command outputs the design files at `build/sky130_erroramp`

In [ ]:
%cd /content/pmicgen/xschem
!xschem -q -r -n --rcfile xschemrc tests/tb_ota.sch
!ngspice -b -a -o /content/pmicgen/build/sky130_erroramp/erroramp.report /root/.xschem/simulations/tb_ota.spice
%cd /content/pmicgen
!pmicgen --tech sky130A ota --netlist align/mabrains_erroramp
!mkdir -p build/sky130_erroramp/gds
!mv MABRAINS_ERRORAMP_0.gds build/sky130_erroramp/gds/erroramp.gds
!mkdir -p build/sky130_erroramp/ngspice/pre_tb
!mv xschem/ota_accm.raw build/sky130_erroramp/ngspice/pre_tb/ota_accm.raw
!mv xschem/ota_acdiff.raw build/sky130_erroramp/ngspice/pre_tb/ota_acdiff.raw
!mv xschem/ota_psrr.raw build/sky130_erroramp/ngspice/pre_tb/ota_psrr.raw

### 4.2.2 Layout
This blocks shows the previously generated layout.

In [ ]:
%cd /content/pmicgen
import gdsfactory as gf
import sky130
pmosw: gf.Component = gf.read.import_gds("build/sky130_erroramp/gds/erroramp.gds")
pmosw.plot_matplotlib()

### 4.2.3 Schematic Simulation

AC analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_erroramp/ngspice/pre_tb/ota_accm.raw',
    delim_whitespace=True,
    header=None,
    names=['mag', 'f', 'phase'])

plt.figure(figsize=(10, 6))

plt.plot(df['f'], df['phase'], linestyle='-', label='Phase')

ax = plt.gca()
ax2 = ax.twinx() 
ax2.plot(df['f'], df['mag'], linestyle='-', color='r', label='dB')

plt.title('AC analysis')
ax.set_xlabel('Frequency')
ax.set_ylabel('Phase (degrees)')
ax2.set_ylabel('dB')

plt.xscale('log')

plt.grid(True)

ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

PSRR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_erroramp/ngspice/pre_tb/ota_psrr.raw',
    delim_whitespace=True,
    header=None,
    names=['mag', 'f', 'phase'])

plt.figure(figsize=(10, 6))

plt.plot(df['f'], df['phase'], linestyle='-', label='Phase')

ax = plt.gca()
ax2 = ax.twinx() 
ax2.plot(df['f'], df['mag'], linestyle='-', color='r', label='dB')

plt.title('PSRR')
ax.set_xlabel('Frequency')
ax.set_ylabel('Phase (degrees)')
ax2.set_ylabel('dB')

plt.xscale('log')

plt.grid(True)

ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

## 4.3 Common Centroid Resistance

The common centroid resistance is generated using a automatic place and route algorithm embedded into the OnGridPnr (https://github.com/pmicgen/on-grid-pnr) sub-repository

### 4.3.1 Generation
The following command outputs the design files at `build/sky130_ccres`

In [ ]:
!pmicgen ccres --tech sky130 --ratio 0.6 --row 5 --col 5

### 4.3.2 Layout
This blocks shows the previously generated layout.

In [ ]:
import gdsfactory as gf
import sky130
pmosw: gf.Component = gf.read_gds("build/sky130_pmosw/pmosw.gds")
gf.plot(pmosw)

### 4.3.3 Schematic Simulation

DC analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ccres/ngspice/pre_tb/dc.csv',
    delim_whitespace=True,
    header=None,
    names=['i', 'v'])
plt.figure(figsize=(10, 6))
plt.plot(df['i'], df['v'], marker='o', linestyle='-')
plt.title('i vs v')
plt.xlabel('v')
plt.ylabel('i')
plt.grid(True)
plt.show()

### 4.4.3 Voltage Divider

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ccres/ngspice/pre_tb/vdiv.csv',
    delim_whitespace=True,
    header=None,
    names=['vout', 'vin'])
plt.figure(figsize=(10, 6))
plt.plot(df['vout'], df['vin'], marker='o', linestyle='-')
plt.title('Voltage divider')
plt.xlabel('vin')
plt.ylabel('vout')
plt.grid(True)
plt.show()

## 4.5 Pass Transistor

The pass transistor is a power PMOS in a waffle layout design extracted from the 3LFCC work.

### 4.5.3 Generation
The following command outputs the design files at `build/sky130_pmosw`

Keep in mind that the multiplicty scales into discretes values, so an albitrary value will be approximated 

In [ ]:
%cd /content/pmicgen
!mkdir -p build/sky130_pmosw/ngspice/pre_tb
!ngspice -b xschem/designs/pmosw/pmosw.spice
!pmicgen --tech sky130A pmosw --mult 480

### 4.5.2 Layout

In [ ]:
import gdsfactory as gf
pmosw: gf.Component = gf.read.import_gds("build/sky130_pmosw/gds/pmosw.gds")
pmosw.plot()

### 4.5.3 Schematic Simulation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_pmosw/ngspice/pre_tb/sky130_pmosw/id_vds.raw',
    delim_whitespace=True,
    header=None,
    names=['id', 'vds'])

plt.figure(figsize=(10, 6))
plt.plot(df['id'], df['vds'], linestyle='-')
plt.title('id vs vds')
plt.xlabel('ids')
plt.ylabel('id')
plt.legend("vgs = 5")
plt.grid(True)
plt.show()

## 5. Low Drop Out Regulator Top Simulations (Work in progress)

### 5.1 Generation
The following commands outputs the design files at `build/sky130_ldo`

This design can be generatede either through a list of specs or through the components generated previously

In [ ]:
%cd /content/pmicgen/xschem
!xschem -q -r -n --rcfile xschemrc tests/ldo_w_jkustin_bgr.sch
%cd /content/pmicgen
!pmicgen --tech sky130A ldo
!mkdir -p build/sky130_bgr/ngspice/pre_tb
!mv xschem/ldo.csv build/sky130_bgr/ngspice/pre_tb

### 5.2 Schematic Simulation

Supply Variation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/pre_tb/supplyvar.csv',
    delim_whitespace=True,
    header=None,
    names=['vs', 'vdd', 'vout'])

plt.figure(figsize=(10, 6))
plt.plot(df['vout'], df['vs'], marker='o', linestyle='-')
plt.plot(df['vdd'], df['vs'], marker='o', linestyle='-')
plt.title('Supply variation')
plt.xlabel('vs')
plt.ylabel('v')
plt.grid(True)
plt.show()

PSRR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/pre_tb/psrr.csv',
    delim_whitespace=True,
    header=None,
    names=['f', 'mag'])

plt.figure(figsize=(10, 6))
plt.plot(df['f'], df['mag'], marker='o', linestyle='-')
plt.title('PSRR')
plt.xlabel('freq')
plt.ylabel('mag')
plt.grid(True)
plt.show()

Load Transient

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/pre_tb/loadtran.csv',
    delim_whitespace=True,
    header=None,
    names=['v', 't'])

plt.figure(figsize=(10, 6))
plt.plot(df['v'], df['t'], marker='o', linestyle='-')
plt.title('Load Transient')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.grid(True)
plt.show()

### Post Layout Simulation

Supply variation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/post_tb/supplyvar.csv',
    delim_whitespace=True,
    header=None,
    names=['vs', 'vdd', 'vout'])

plt.figure(figsize=(10, 6))
plt.plot(df['vout'], df['vs'], marker='o', linestyle='-')
plt.plot(df['vdd'], df['vs'], marker='o', linestyle='-')
plt.title('Supply variation')
plt.xlabel('vs')
plt.ylabel('v')
plt.grid(True)
plt.show()

PSRR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/post_tb/psrr.csv',
    delim_whitespace=True,
    header=None,
    names=['f', 'mag'])

plt.figure(figsize=(10, 6))
plt.plot(df['f'], df['mag'], marker='o', linestyle='-')
plt.title('PSRR')
plt.xlabel('freq')
plt.ylabel('mag')
plt.grid(True)
plt.show()

Load Transient

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('build/sky130_ldo/ngspice/post_tb/loadtran.csv',
    delim_whitespace=True,
    header=None,
    names=['v', 't'])

plt.figure(figsize=(10, 6))
plt.plot(df['v'], df['t'], marker='o', linestyle='-')
plt.title('Load Transient')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.grid(True)
plt.show()

## 15. 3 - Level Flying Capacitor Converter
Section work in progress

### Generation
The following command outputs the design files at `build/sky130_3lfcc`

In [ ]:
!pmicgen 3lfcc --tech sky130

## 16. PMIC

### Generation
The following command outputs the designs files at `build/sky130_pmic`

In [ ]:
!pmicgen pmic --tech sky130

## 17. References

[1] Kose, Selçuk, et al. "Active filter-based hybrid on-chip DC–DC converter for point-of-load voltage regulation." IEEE Transactions on Very Large Scale Integration (VLSI) Systems 21.4 (2012): 680-691.

[2] Vaisband, Inna, and Eby G. Friedman. "Heterogeneous methodology for energy efficient distribution of on-chip power supplies." IEEE Transactions on Power Electronics 28.9 (2012): 4267-4280.

[3] Marin, Jorge, et al. "Design and Automated Layout Generation of a PMIC Core in Skywater 130nm Open-Source Technology." 2024 IEEE 15th Latin America Symposium on Circuits and Systems (LASCAS). IEEE, 2024.

[4] Mohamed, Karimeldeen, Sherif Nafea, and Hesham Omran. "Design automation of low dropout voltage regulators: A general approach." Electronics 12.1 (2022): 205.

[5] AC3E Microelectronics Team, "Automated Generation of Power Transistors and 3LFC DC-DC Converter", https://github.com/sscs-ose/sscs-ose-code-a-chip.github.io/blob/main/VLSI23/accepted_notebooks/3LFCC/3LFCC_v2p0.ipynb

[6] Chen, Yue, et al. "A supply pushing reduction technique for LC oscillators based on ripple replication and cancellation." IEEE Journal of Solid-State Circuits 54.1 (2018): 240-252.
